# Implementation of Linear Regression on a Large Dataset Using Dask Library

## Objectives

- understand how dask handles large dataset over pandas dataframe 
- perform exploratory data analysis on a large dataset (2 Million rows) using dask
- implement linear regression model using dask library and make predictions


## Problem Statement

 Predict the taxi fare amount in New York city using Dask-ML.

Note that the features the ' Dask_MP_dataset.csv' file that is provided for this miniproject varies slightly in terms of included features, as compared to the original dataset described above. Please proceed with the provided csv file.

In [ ]:
#@title Install Dask dependencies and restart runtime
!pip -qq install dask-ml
!pip -qq install dask
!pip -qq install dask[complete]
!pip -qq install dask distributed
!pip -qq install mimesis

#### Importing Necessary Packages

In [ ]:
import warnings
warnings.filterwarnings('ignore')
import dask
import dask.dataframe as dd
import dask.array as da
from dask_ml.linear_model import LinearRegression
from dask_ml.model_selection import train_test_split
from dask_ml.metrics import mean_squared_error, r2_score
from dask.distributed import Client
import time as time
import numpy as np
import pandas as pd
import seaborn as sns, matplotlib.pyplot as plt
%matplotlib inline

from dask.distributed import Client, progress
# client = Client()

In [1]:
#@title Download the data

#### Read the dataset using dask library and compare the time of execution with pandas library.

In [ ]:
%%time
ddf = dd.read_csv('/content/Dask_MP_dataset.csv', dtype={'passenger_count': 'int64'})

#### Use pandas to read the dataset and compare the time taken

In [ ]:
%%time
df_pd = pd.read_csv('/content/Dask_MP_dataset.csv')

In [ ]:
%%time
ddf.head()

### Data Analysis

#### Drop the unnecessary columns. Also drop the duplicate rows and the rows having null values.

In [ ]:
ddf = ddf.drop(["key", "Unnamed: 0"],axis=1)

In [ ]:
# Drop duplicate rows
%%time
ddf = ddf.drop_duplicates()

In [ ]:
# ddf['pickup_datetime'] = dask.dataframe.to_datetime(ddf['pickup_datetime'])
# ddf['year'] = ddf['pickup_datetime'].dt.year
# ddf.head()

In [ ]:
%%time
ddf = ddf.dropna()

CPU times: user 6.04 ms, sys: 148 µs, total: 6.18 ms
Wall time: 6.88 ms


In [ ]:
# Drop duplicate rows in pandas dataframe
%%time
df_pd = df_pd.drop_duplicates()

In [ ]:
%%time
df_pd = df_pd.dropna()

In [ ]:
%%time
ddf.groupby("passenger_count").fare_amount.mean().compute()

In [ ]:
%%time
df_pd.groupby("passenger_count").fare_amount.mean()

CPU times: user 33.6 ms, sys: 67 µs, total: 33.6 ms
Wall time: 39.6 ms


passenger_count
0        8.862578
1       11.224824
2       11.819826
3       11.533967
4       11.747775
5       11.208924
6       12.158254
9      104.000000
208      3.300000
Name: fare_amount, dtype: float64

In [ ]:
%%time 
ddf[["fare_amount"]].mean().compute()

In [ ]:
%%time 
df_pd[["fare_amount"]].mean()

#### Visualize the target variable, i.e., `fare_amount` to study the fare distribution, using a histogram density plot. Analyze the fare_amount distribution, try to visualize it for a range of [0, 60].


In [ ]:
#exploring data

def plot_dist(series=ddf["fare_amount"], title = "Fare Distribution"):
  sns.histplot(series, kde=True, stat='density',discrete=True)
  sns.despine()
  plt.title(title)
  sns.histplot(series, kde=True, stat='density',discrete=True)
  sns.despine()
  plt.title(title);
  plt.show();
  plt.show()
plot_dist()

In [ ]:
#dropping absurd values and plotting fare amount in the range [0, 60]
ddf = ddf[ddf.fare_amount.between(0,60)]
plot_dist(ddf.fare_amount)

#### Observe the number of workers and cores running in your machine

Initialize a client and observe how many workers are working and the number of cores utilizing for the given data set.

In [ ]:
# #Initializing a client
# client = Client(processes=False)
# client

### EDA based on Time

#### Extract day of the week (dow), hour, month and year from `pickup_datetime`.

In [ ]:
# pickup_datetime feature

def extract_time_features(ddf):
    timezone_name = 'America/New_York'
    time_column = "pickup_datetime"
    ddf.index = pd.to_datetime(ddf[time_column])
    #ddf.index = ddf.index.tz_convert(timezone_name)
    ddf["dow"] = ddf.index.weekday
    ddf["hour"] = ddf.index.hour
    ddf["month"] = ddf.index.month
    ddf["year"] = ddf.index.year
    return ddf.reset_index(drop=True)

In [ ]:
ddf = extract_time_features(ddf.compute())

In [ ]:
ddf.head()

#### Plot the taxi trip by hour of the day

* Partition the data into segments using `dask.from_pandas()`

* Plot the taxi trip for hour of the day.

In [ ]:
ddf = dd.from_pandas(ddf, npartitions=3)

In [ ]:
type(ddf)

In [ ]:
# taxi trip repartition by hour of the day

sns.catplot(x="hour", kind="count", palette="icefire", data=ddf.compute(), height=5, aspect=3);
sns.despine()
plt.title('Hour of Day');
plt.show()

#### Plot the taxi trip repartition by day of the week (dow)

In [ ]:
# taxi trip repartition by day of the week

sns.catplot(x="dow", kind="count", palette="icefire", data=ddf.compute(), height=5, aspect=3);
sns.despine()
plt.title('Day of Week');
plt.show()

#### Draw a plot between the target variable and passenger count and analyze it. 

In [ ]:
#passenger count feature

sns.catplot(x="passenger_count", y="fare_amount", palette="icefire", data=ddf.compute(), kind="bar", aspect=3)
sns.despine()
plt.show()

#### Draw a plot between the target variable and hour and analyze it. 

In [ ]:
#fare amount by hour

sns.catplot(x="hour", y="fare_amount", palette="icefire", data=ddf.compute(), kind="bar", aspect=3)
sns.despine()
plt.show()

#### Compute the Haversine distance between samples

* Convert the latitude and longitude co-rodinates to radians

* Calculate the Haversine distance

  [haversine_distances](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.pairwise.haversine_distances.html)

* Add the "distance" feature to the dataset and plot its distribution

In [ ]:
# Distance feature (given formula)

def haversine_distance(ddf,
                       start_lat="start_lat",
                       start_lon="start_lon",
                       end_lat="end_lat",
                       end_lon="end_lon"):
    
    # Calculate the great circle distance between two points 
    #on the earth (specified in decimal degrees).
       
    # Vectorized version of the haversine distance for pandas df
    #Computes distance in kms
    
    lat_1_rad, lon_1_rad = np.radians(ddf[start_lat].astype(float)), np.radians(ddf[start_lon].astype(float))
    lat_2_rad, lon_2_rad = np.radians(ddf[end_lat].astype(float)), np.radians(ddf[end_lon].astype(float))
    dlon = lon_2_rad - lon_1_rad
    dlat = lat_2_rad - lat_1_rad

    a = np.sin(dlat / 2.0) ** 2 + np.cos(lat_1_rad) * np.cos(lat_2_rad) * np.sin(dlon / 2.0) ** 2
    c = 2 * np.arcsin(np.sqrt(a))
    haversine_distance = 6371 * c
    return haversine_distance

In [ ]:
%%time
ddf["distance"] = haversine_distance(ddf, 
                                     start_lat="pickup_latitude", start_lon="pickup_longitude",
                                     end_lat="dropoff_latitude", end_lon="dropoff_longitude"
                                     )

In [ ]:
ddf.distance.describe()

In [ ]:
%matplotlib inline
plot_dist(series=ddf[ddf.distance<50].distance, title = "Distance distribution")

### Correlation between distance and fare amount

In [ ]:
# Correlation between fare_amount and distance

sns.scatterplot(x="distance", y="fare_amount", palette="icefire",data=ddf[ddf.distance < 80].compute().sample(10000))
plt.show()

### Preparing dataset for model implementation

In [ ]:
# Read the dataset to prepare for training

data_train = ddf

### Removing outliers from training set Based on Coordinates

#### Remove the outliers using the given latitude and longitude features from the dataset. We need to analyze the data of taxi within New York City.

Given the co-ordinates of New York city are Latitude: 40.7128° and Longitude: -74.0060°. We can include the pickup and drop off points such that there left and right value mean will be the given co-ordinate value. 

In [ ]:
%%time
data_train = data_train[data_train["pickup_latitude"].between(left = 40, right = 42 )]
data_train = data_train[data_train["pickup_longitude"].between(left = -74.3, right = -72.9 )]
data_train = data_train[data_train["dropoff_latitude"].between(left = 40, right = 42 )]
data_train = data_train[data_train["dropoff_longitude"].between(left = -74, right = -72.9 )]

In [ ]:
%%time
data_train["distance"] = haversine_distance(data_train, 
                                      start_lat="pickup_latitude", start_lon="pickup_longitude",
                                      end_lat="dropoff_latitude", end_lon="dropoff_longitude"
                                     )

In [ ]:
data_train.head()

#### Divide the data into train and test splits with X as feature variables and y as target variable

* Divide data into train test split with 70-30 ratio

In [ ]:
X = data_train.drop(["fare_amount", "pickup_datetime"], axis=1)
y = data_train[["fare_amount"]]

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=0)

In [ ]:
type(X_train)

In [ ]:
type(y_train)

In [ ]:
X_train = X_train.compute()

In [ ]:
y_train = y_train.compute()

In [ ]:
#fit the model
lr = LinearRegression()

In [ ]:
%%time
lr.fit(X_train.values, y_train.values)

#### Predict the test data and calculate the mean squared error and r2 score.

In [ ]:
%%time
X_test = X_test.compute()
y_pred = lr.predict(X_test.values)

In [ ]:
y_pred

In [ ]:
y_test = y_test.to_dask_array(lengths=True)


In [ ]:
y_test = y_test.reshape(-1)

In [ ]:
y_test.compute()

In [ ]:
# Mean squared error
%%time
mean_squared_error(y_test, y_pred)

In [ ]:
# R2_Score
%%time
r2_score(y_test, y_pred)